# Case 3: User KGI 04

## Import Dependencies

In [1]:
import pandas as pd

## Data Loading

In [2]:
PATH = './Dataset/MiningProcess_Flotation_Plant_Database.csv'

df = pd.read_csv(PATH, parse_dates=['date'])
print("Dataset loaded successfully!")

Dataset loaded successfully!


## Basic Data Exploration

In [3]:
print("\n--- Data Exploration ---")
print(f"Dataset Shape: {df.shape}")
print("First 5 rows:")
print(df.head())

print("\nBasic Statistics:")
print(df.describe())

print("\nMissing Values:")
missing_values = df.isnull().sum()
print(missing_values)
print(f"Missing Values Percentage:")
print((missing_values / len(df)) * 100)


--- Data Exploration ---
Dataset Shape: (737453, 24)
First 5 rows:
                 date % Iron Feed % Silica Feed Starch Flow Amina Flow  \
0 2017-03-10 01:00:00        55,2         16,98     3019,53    557,434   
1 2017-03-10 01:00:00        55,2         16,98     3024,41    563,965   
2 2017-03-10 01:00:00        55,2         16,98     3043,46    568,054   
3 2017-03-10 01:00:00        55,2         16,98     3047,36    568,665   
4 2017-03-10 01:00:00        55,2         16,98     3033,69    558,167   

  Ore Pulp Flow Ore Pulp pH Ore Pulp Density Flotation Column 01 Air Flow  \
0       395,713     10,0664             1,74                      249,214   
1       397,383     10,0672             1,74                      249,719   
2       399,668      10,068             1,74                      249,741   
3       397,939     10,0689             1,74                      249,917   
4       400,254     10,0697             1,74                      250,203   

  Flotation Column 02 Ai